In [ ]:
# If necessary, install packages
!pip install -r requirements.txt

In [ ]:
import gdown
import os
import difflib
import pandas as pd

In [ ]:
# Function to match twitter accounts with bill sponsors
def match_sponsor_simple(name_clean):
    choices = historical_df["normalized_name"].tolist() # tweet dataframe
    match = difflib.get_close_matches(name_clean, choices, n=1, cutoff=0.7)
    if match:
        match_row = historical_df[historical_df["normalized_name"] == match[0]].iloc[0]
        score = difflib.SequenceMatcher(None, name_clean, match[0]).ratio()
        return pd.Series({
            "matched_name": match_row["matched_name"],
            "twitter_screen_name": match_row["screen_name"],
            "twitter_id": match_row["twitter_id"],
            "match_score": round(score * 100, 1),
            "match_method": "fuzzy",
            "flag_review": score < 0.85
        })
    return pd.Series({
        "matched_name": None,
        "twitter_screen_name": None,
        "twitter_id": None,
        "match_score": None,
        "match_method": None,
        "flag_review": True
    })

# Apply matching with flagging
matched_info_simple = sponsor_df["normalized_name_clean"].apply(match_sponsor_simple) # sponsor dataframe
final_matches_flagged = pd.concat([sponsor_df["raw_name"], matched_info_simple], axis=1)
